# Расчет выйгрыша на каждом клиенте

In [3]:
import pandas as pd

# данные о каждом из клиентов
data = pd.read_csv(
    "/home/dranik/KFA/university/input_data_diprom.csv", 
    na_values = ['-', 'нет информации'],
    parse_dates = [
        'Автомобиль год выпуска1',
        'Автомобиль год выпуска2',
        'Автомобиль год выпуска3',
        'Дата регистрации договора',
        'Дата рождения',
        'Дата планируемого закрытия',
        'Дата фактического закрытия'
    ],
    low_memory = False
)

# данные о процентных ставках
interest_rates = pd.read_excel(
    "/home/dranik/KFA/university/diplom_project/prog/interest rates/interest_rantes.xlsx",
    index_col = 0)

# Вычислительные функции

In [39]:
def get_int_rate(reg_date, term):
    '''Получение процентной ставки для выбранного клиента'''
    int_rate_ind = \
        (reg_date > interest_rates.index).sum()
    
    possible_rates = interest_rates.iloc[int_rate_ind - 1]
    
    if term < 365:
        rate = possible_rates[0]
    else:
        rate = possible_rates[1]
        
    return rate

def comput_vaits_return(def_inf, PV, FV):
    '''
        Вычисление того сколько ожидается возврата от клиента.
        Предполагается, что любой клиент с просрочкой ниже 60 дн.
        рано или поздно вернет кредит потому это будет наращенная сумма.
        Для "безнадежных" клиентов предполагаеются потери в размере
        разницы суммы того, что он должен был вернуть и успел вернуть.
    '''
    if def_inf <= 60:
        return FV - PV
    
    
    #client_info[]

In [56]:
from datetime import date, timedelta
date(2021, 1,1) - timedelta(days = 5)

datetime.date(2020, 12, 27)

# Применяем вычисления ко всему множеству строк в DataFrame
Удалив только наблюдения, что имеют пропуск в "дата пранируемого закрытия" - это почти всегда "Овердрафт" и сохраняем результат.

In [45]:
data = data.dropna(subset = "Дата планируемого закрытия")

data['Срок кредита дн.'] = \
    (data["Дата планируемого закрытия"] - data["Дата регистрации договора"]).\
    astype('timedelta64[D]')
data['Процентная ставка'] = data.apply(
    lambda client_info: get_int_rate(client_info["Дата регистрации договора"],
                                    client_info["Срок кредита дн."]),
    axis = 1)
data["Сумма к возврату"] = \
    data['Сумма договора']*(1 + data['Процентная ставка']/36000)**\
    data['Срок кредита дн.']


# # compute_profit(data.iloc[1])
# # data.iloc[1]["Сумма договора"]
# data["Сумма возврата"] = data.apply(compute_profit, axis = 1)

# data[["Сумма договора", "Сумма возврата"]].\
#     to_csv("/home/dranik/KFA/university/proc_ret_sum.csv")